In [370]:
import os
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit


importlib.reload(setup)
importlib.reload(pipeline)
importlib.reload(setup_overlap)
importlib.reload(setup_bestfit)
importlib.reload(pipeline_lblnew)
importlib.reload(bestfit)

<module 'climatools.lblnew.bestfit_params' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/bestfit_params.py'>

In [371]:
def molbands():
    return [('h2o', '3a'),
            ('co2', '3a'),
            ('n2o', '3a')]
#    return [('h2o', '1')]
#                ('h2o', '2'),
#                ('h2o', '3a'),
#                ('h2o', '3b'),
#                ('h2o', '3c'),
#                ('h2o', '4'),
#                ('h2o', '5'),
#                ('h2o', '6'),
#                ('h2o', '7'), 
#                ('h2o', '8'),
#                ('h2o', '9'),
#                ('co2', '3a'), 
#                ('co2', '3b'),
#                ('co2', '3c'),
#                ('co2', '9'),
#                ('o3', '5'),
#                ('o3', '9'),
#                ('n2o', '3a'),
#                ('n2o', '7'),
#                ('ch4', '7') ]



def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
                     '2': '2',
                     '3a': '3',
                     '3b': '4',
                     '3c': '5',
                     '4': '6', 
                     '5': '7',
                     '6': '8',
                     '7': '9', 
                     '8': '10',
                     '9': '11'}
    
    return lblnew2clirad

In [372]:
def param_lblnew(molecule=None, band=None):
    param = bestfit.kdist_params(molecule=molecule, band=band)
    param['option_compute_ktable'] = 0
    return param


def param_cliradlw(molecule=None, band=None):
    
    param = bestfit.kdist_params(molecule=molecule, band=band)

    clirad_band = int(band_map()[band])
    conc = 'atmpro' if param['conc'] == None else param['conc']
    
    return {'atmpro': param['atmpro'],
            'band': [clirad_band],
            'commitnumber': '7fe1447',
            'molecule': {molecule: conc},
            'tsfc': param['tsfc']}

def parampairs():
    ps = []
    
    for molecule, band in molbands():
        for atmpro, tsfc in [('mls', 294), ('saw', 257), ('trp', 300)]:
            p_lblnew = param_lblnew(molecule=molecule, band=band)
            p_clirad = param_cliradlw(molecule=molecule, band=band)
            p_lblnew['atmpro'] = atmpro
            p_lblnew['tsfc'] = tsfc
            p_clirad['atmpro'] = atmpro
            p_clirad['tsfc'] = tsfc
        
            ps.append((p_clirad, p_lblnew))
    
    return ps


############ 

#def param_lblnew_overlap():
#    return {'atmpro': 'mls',
#            'band': '3a',
#            'commitnumber': '5014a19',
#            'dv': 0.01,
#            'molecule': {'co2': 0.0004, 
#                         'h2o': 'atmpro', 
#                         'n2o': 3.2e-07},
#            'nv': 100,
#            'tsfc': 294}
#
#def param_cliradlw():
#    p_lblnew = param_lblnew_overlap()
#    clirad_band = int(band_map()[p_lblnew['band']])
#    
#    return {'atmpro': 'mls',
#            'band': [clirad_band],
#            'commitnumber': 'fc273c9',
#            'molecule': p_lblnew['molecule'],
#            'tsfc': 294}
#
#
#
#def parampairs():
#    ps = []
#    for atmpro, tsfc in [('mls', 294), ('saw', 257), ('trp', 300)]:
#        p_clirad = param_cliradlw()
#        p_lblnew = param_lblnew_overlap()
#        p_clirad['atmpro'] = atmpro
#        p_clirad['tsfc'] = tsfc
#        p_lblnew['atmpro'] = atmpro
#        p_lblnew['tsfc'] = tsfc
#        
#        ps.append((p_clirad, p_lblnew))
#    
#    return ps

##############

def script():
    ps = parampairs()
    gprocs = pipeline.pipeline_fortran2ipynb2git(parampairs=ps,
                                                 setup=setup)
    
    for param, _ in ps:
        print(param['molecule'], 
              ' band', param['band'], 
              param['atmpro'])
        print(pipeline.nbviewer_url(param=param, setup=setup))
        print()

In [373]:
script()

Submitting radiation calculation for cases
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'n2o': 3.2e-07},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'n2o': 3.2e-07},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [3],
 'commitnumber': '7fe1447',
 'molecule': {'n2o': 3.2e-07},
 'tsfc': 300}

Submitting 